# FPGA Implementation of a Simple Neural network 

## Load required packages and datasets

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

### Perform Normalization and train-test split

In [3]:
# load the breast cancer dataset
X, y = load_breast_cancer(return_X_y=True)
# split the train and test dataset
X_train, X_test,\
	y_train, y_test = train_test_split(X, y,
									test_size=0.20,
									random_state=23)
#feature Scaling  
from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
X_train= st_x.fit_transform(X_train)    
X_test= st_x.transform(X_test)  

### Importing Keras module of Deep learning

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout


### Build the model
 **A simple fully connected Neural network contains 16 Neuron at 1st layer with _Relu_ activation, 8 neuron in 2nd layer with _Relu_ activation, Final/output layer having one Neuron with _Sigmoid_ activation**

In [5]:
# Initialising the ANN
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(16, activation='relu', input_shape=(30,)))
# Adding dropout to prevent overfitting
#classifier.add(Dropout(0.1))
# Adding the second hidden layer
classifier.add(Dense(8, activation='relu'))
# Adding dropout to prevent overfitting
#classifier.add(Dropout(0.1))
# Adding the output layer
classifier.add(Dense(1, activation='sigmoid'))
# Compiling the ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size=100, epochs=150)

Epoch 1/150
5/5 [==============================] - 1s 3ms/step - loss: 0.6615 - accuracy: 0.6462
Epoch 2/150
5/5 [==============================] - 0s 2ms/step - loss: 0.6127 - accuracy: 0.6747
Epoch 3/150
5/5 [==============================] - 0s 2ms/step - loss: 0.5713 - accuracy: 0.7253
Epoch 4/150
5/5 [==============================] - 0s 2ms/step - loss: 0.5348 - accuracy: 0.7626
Epoch 5/150
5/5 [==============================] - 0s 2ms/step - loss: 0.4997 - accuracy: 0.8066
Epoch 6/150
5/5 [==============================] - 0s 2ms/step - loss: 0.4678 - accuracy: 0.8418
Epoch 7/150
5/5 [==============================] - 0s 2ms/step - loss: 0.4373 - accuracy: 0.8615
Epoch 8/150
5/5 [==============================] - 0s 2ms/step - loss: 0.4098 - accuracy: 0.8791
Epoch 9/150
5/5 [==============================] - 0s 2ms/step - loss: 0.3824 - accuracy: 0.8901
Epoch 10/150
5/5 [==============================] - 0s 2ms/step - loss: 0.3567 - accuracy: 0.9055
Epoch 11/150
5/5 [===========

**Predict the result**

In [6]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_keras=classifier.predict(X_test)
y_pred = (y_pred > 0.5)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

4/4 [==============================] - 0s 2ms/step
[[38  1]
 [ 1 74]]


In [7]:
print(y_test)

[1 1 0 1 0 0 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 1 1
 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 0 1 0 0 1 0 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1
 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 0 0 0 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 1 0 1
 1 1 0]


In [8]:
from sklearn.metrics import accuracy_score
print('Accuracy score is:',100*accuracy_score(y_test, y_pred),'%')

Accuracy score is: 98.24561403508771 %


## Export Weight and Bias of trained model

In [9]:
#print model parameters: weight and bias
first_layer_weights = classifier.layers[0].get_weights()[0]
first_layer_biases  = classifier.layers[0].get_weights()[1]
second_layer_weights = classifier.layers[1].get_weights()[0]
second_layer_biases  = classifier.layers[1].get_weights()[1]
third_layer_weights = classifier.layers[2].get_weights()[0]
third_layer_biases  = classifier.layers[2].get_weights()[1]

In [10]:
print(first_layer_biases)

[ 0.22651652 -0.00651723  0.16685447  0.27603388  0.2276782   0.3001223
 -0.14324668  0.23136708  0.22145116  0.10626026  0.23359351  0.0282598
  0.2324835   0.26536405  0.03963194  0.24475598]


**Fetch weight and bias of each layer of trained model and store them into Numpy array, then multiply Quantization factor $2^{12}$ and after rounding, save them as a separate C file** 

In [11]:
import numpy as np
#export model parameters for 1st layer
layer0_w = first_layer_weights
layer0_b = first_layer_biases
n_features = 30 #no of features as input
n_layer0_neuron = 16 #no of neuron in layer0
layer0_w = pow(2,12)*(layer0_w.T)
layer0_b = pow(2,12)*(layer0_b)
f=open('model_params_layer0.c','w')
#write weights of layer_0 into test file
f.write("int w0[16][30]= {")
for j in range(n_layer0_neuron):
    f.write("{")
    for i in range(n_features):
        f.write(str(round(layer0_w[j][i])))
        if(i<n_features - 1):
            f.write(",")
        if((i+1)%10==0):
            f.write("\n")
    f.write("}")
    if(j<n_layer0_neuron -1 ):
        #print(j)
        f.write(",")
f.write("};"+"\n\n")
#write bias values into file
f.write("int b0[16] = {")
for i in range(n_layer0_neuron):
    f.write(str(round(layer0_b[i])))
    if(i< n_layer0_neuron -1):
        f.write(",")
    if((i+1)%10==0):
        f.write("\n")
   
f.write("};"+"\n\n")
f.close()

#export model parameters for 2nd layer
layer1_w = second_layer_weights
layer1_b = second_layer_biases
n_features = n_layer0_neuron #no of features as input i.e o/p of prev layer
n_layer1_neuron = 8 #no of neuron in layer0
layer1_w = pow(2,12)*(layer1_w.T)
layer1_b = pow(2,12)*(layer1_b)
f=open('model_params_layer1.c','w')
#write weights of layer_1 into test file
f.write("int w1[8][16]= {")
for j in range(n_layer1_neuron):
    f.write("{")
    for i in range(n_features):
        f.write(str(round(layer1_w[j][i])))
        if(i<n_features - 1):
            f.write(",")
        if((i+1)%10==0):
            f.write("\n")
    f.write("}")
    if(j<n_layer1_neuron -1):
        #print(j)
        f.write(",")
f.write("};"+"\n\n")
#write bias values into file
f.write("int b1[8] = {")
for i in range(n_layer1_neuron):
    f.write(str(round(layer1_b[i])))
    if(i< n_layer1_neuron -1):
        f.write(",")
    if((i+1)%10==0):
        f.write("\n")
   
f.write("};"+"\n\n")
f.close()

#export model parameters for 3rd layer
layer2_w = third_layer_weights
layer2_b = third_layer_biases
n_features = n_layer1_neuron #no of features as input i.e o/p of prev layer
n_layer2_neuron = 1 #no of neuron in layer0
layer2_w = pow(2,12)*(layer2_w.T)
layer2_b = pow(2,12)*(layer2_b)
f=open('model_params_layer2.c','w')
#write weights of layer_2 into test file
f.write("int w2[8]= ")
for j in range(n_layer2_neuron):
    f.write("{")
    for i in range(n_features):
        f.write(str(round(layer2_w[j][i])))
        if(i<n_features - 1):
            f.write(",")
        if((i+1)%10==0):
            f.write("\n")
    
f.write("};"+"\n\n")
#write bias values into file
f.write("int b2 = ")
for i in range(n_layer2_neuron):
    f.write(str(round(layer2_b[i])))
    if(i< n_layer2_neuron -1):
        f.write(",")
    if((i+1)%10==0):
        f.write("\n")
   
f.write(";"+"\n\n")
f.close()

**Similarly save any one test feaure into _feature.c_ file for testing**

In [12]:
#Convert test features to fixed point for verification
f = open("feature.c","w")
f.write("int x[30] = {")
for i in range(30):
    f.write(str(round((pow(2,12)*X_test[0][i]))))
    if(i<29):
            f.write(",")
    if((i+1)%10==0):
            f.write("\n")
    
f.write("};"+"\n\n")
f.close()
print("Actual label for this input feature:",y_test[0])

Actual label for this input feature: 1


### Check the current working directory where this script is located. Few C source code file is generated. They contain weight and bias values of different layer. Now create a C code to implement the same network in fixed point representation and include those source files into that project